In [ ]:
%pip install openai
%pip install --upgrade langchain
%pip install langchain-community

In [ ]:
# Setup environment and import required libraries
import openai

In [ ]:
%set_env OPENAI_API_KEY=#Your API Key here

In [ ]:
llm_model="gpt-3.5-turbo"

Neste momento já temos o necessário para comunicar com a API da OpenAI.
Para avançar, vamos criar uma função auxiliar que irá completar as frases que enivemos nas chamadas:

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

## Prompt
Vamos adaptar o caso de uso realizado para classificar o tópico em um atendimento.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# Note que as linhas sobre formatação foram intencionalmente
# removidas quando comparadas à mensagem original do lab_01

system_message = f"""
Você vai receber perguntas de atendimento a clientes. \
Classifique cada pergunta em uma categoria primária \
e uma categoria secundária.

Categorias Primárias:
1. Hardware
2. Software
3. Rede e Conectividade
4. Segurança
5. Suporte e Manutenção

Categorias secundárias de 1. Hardware:
1.1: Computadores
1.2: Periféricos
1.3: Componentes Internos

Categorias secundárias de 2. Software:
2.1: Sistemas Operacionais
2.2: Aplicativos
2.3: Sistemas Internos

Categorias secundárias de 3. Rede e Conectividade:
3.1: Internet
3.2: Intranet
3.3: Telefonia e VoIP

Categorias secundárias de 4. Segurança:
4.1: Acesso
4.2: Segurança de Dados
4.3: Segurança Física

Categoria secundárias 5. Suporte e Manutenção
5.1: Atualizações
5.2: Manutenção Preventiva
5.3: Reparos
"""

In [ ]:
user_input_template = "Pergunta de atendimento: {input}"

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", user_input_template)
])

In [ ]:
input="meu teclado nao está funcionando. eu consigo acessar usando um teclado virtual mas as teclas físicas nao funcionam"

In [ ]:
messages = prompt.format_messages(input=input)

In [ ]:
response = chat(messages)
print(response.content)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
class ClassExample(BaseModel):
    primaria: str = Field(description="Categoria Primária")
    secundaria: str = Field(description="Categoria Secundária, identificada como subcategoria da primária.")

# Parte 2 - Parser

In [ ]:
output_parser = JsonOutputParser(pydantic_object=ClassExample)

In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
user_input_template_formatted = """\
Pergunta de atendimento: {input} \
{format_instructions}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", user_input_template_formatted)
])

messages = prompt.format_messages(input=user_input, format_instructions=format_instructions)

In [ ]:
print(messages[1].content)

In [ ]:
response = chat(messages)
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)
print(output_dict)

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('primaria')

## Chains

Um dos pontos mais importantes do Langchain. Permite conectar outros elementos da ferramenta em cadeia para processar o conteúdo.
O poder delas é que podem ser executadas sobre várias entradas (inputs).

Em nosso exemplo anterior toda a execução pode ser resumida em uma única cadeia de prompt, modelo e transcrição de saída.

In [ ]:
chain = prompt | chat | output_parser

In [ ]:
chain.invoke({
    "input": "meu teclado nao está funcionando. eu consigo acessar usando um teclado virtual mas as teclas físicas nao funcionam",
    "format_instructions": format_instructions
})